In [ ]:
# Link https://www.youtube.com/watch?v=lPlJR1xVF8c
# https://github.com/AIAnytime/Qwen2-VL-for-OCR-VQA

In [12]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%cd /content/gdrive/MyDrive/gen-ai/vison-llm


/content/gdrive/MyDrive/gen-ai/vison-llm


In [5]:
pwd

'/content/gdrive/MyDrive/gen-ai/vison-llm'

In [6]:
!python -m pip install git+https://github.com/huggingface/transformers==4.42.4

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vv33cnk5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vv33cnk5
  Resolved https://github.com/huggingface/transformers to commit 6b550462139655d488d4c663086a63e98713c6b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10719728 sha256=b9c37d5cd80c94a1b08eba5bbd31be2be435bb816aa0bc1a6b0d43853b260f5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-d9wjh758/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2


In [7]:
from transformers import Qwen2VLForConditionalGeneration,AutoTokenizer,AutoProcessor

In [9]:
model=Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto")

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [10]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [11]:
from PIL import Image
import requests

In [26]:
# image_path="images/elect2025.png"
# image = Image.open(image_path)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#             },
#             {
#                 "type": "text",
#                 "text": "Explain the election results in details? Your analysis should help the election startegic planner."
#             }
#         ]
#     }
# ]

In [33]:
# image_path="images/form2.png"
# image = Image.open(image_path)

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#             },
#             {
#                 "type": "text",
#                 "text": "You are an expert in creating field configuration from the image? The assistant identifies and extracts different types of form fields such as textboxes, labels, dropdowns, checkboxes, etc., along with their corresponding names or identifiers. The result should be returned in JSON format, with each field type paired with its name or label. The field types may include 'textbox', 'label', 'dropdown', 'checkbox', etc., and the field names are the corresponding headers or identifiers next to the fields. Do not generate your own field names.Your analysis should help the configuration."
#             }
#         ]
#     }
# ]

In [56]:
image_path="images/bar1.png"
image = Image.open(image_path)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "Explain the bar chart in details? Explain each bar in detail with insights of each bar. Your analysis should help the experts in their domain."
            }
        ]
    }
]

In [57]:
text_prompt=processor.apply_chat_template(messages,add_generation_prompt=True)

In [58]:
inputs=processor(images=[image],
                 text=[text_prompt],
                 padding = True,
                 return_tensors="pt")

In [59]:
inputs = inputs.to("cuda")

In [60]:
output_ids = model.generate(**inputs, max_new_tokens=1024)

In [62]:
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [63]:
output_text

['This bar chart compares the usage of various tools and techniques by respondents in the field of business intelligence and analytics. The x-axis measures the percentage of respondents, ranging from 0 to 300. The y-axis represents the different tools and techniques, with a range of options including "Actuate," "SAP," "IBM," "Information Builders," "MicroStrategy," "Microsoft," "SAS," "Panorama Software," "Arcplan," "Target," "Infor," "Board," "IDS Scheer," "LogiXML," "Jaspersoft," "Tibco," "Oracle," and "QlikTech."\n\nThe chart shows that the most commonly used tool is "Actuate," which is used by 150% of respondents. This is followed by "SAP," which is used by 140% of respondents. "IBM" is used by 130% of respondents, and "Information Builders" is used by 120% of respondents. "MicroStrategy" is used by 110% of respondents, and "Microsoft" is used by 100% of respondents. "SAS" is used by 90% of respondents, and "Panorama Software" is used by 80% of respondents. "Arcplan" is used by 70%

In [32]:
output_text

["The exit poll for Delhi's 2025 election, conducted by Axis My India, shows a clear division between different occupational groups and political alliances. Here's a detailed analysis of the results:\n\n### Occupation-wise Vote Share\n#### BJP+\n- Unemployed: 44%\n- Student: 47%\n- Labour: 42%\n- Skilled Professional: 49%\n- Small Shop: 52%\n- Housewife: 45%\n- Private Service: 51%\n- Government Service: 48%\n- Others: 52%\n\n#### AAP\n- Unemployed: 46%\n- Student: 44%\n- Labour: 47%\n- Skilled Professional: 41%\n- Small Shop: 34%\n- Housewife: 47%\n- Private Service: 39%\n- Government Service: 38%\n- Others: 36%\n\n#### CONG\n- Unemployed: 8%\n- Student: 7%\n- Labour: 8%\n- Skilled Professional: 7%\n- Small Shop: 10%\n- Housewife: 5%\n- Private Service: 7%\n- Government Service: 9%\n- Others: 9%\n\n#### Others\n- Unemployed: 2%\n- Student: 2%\n- Labour: 3%\n- Skilled Professional: 3%\n- Small Shop: 4%\n- Housewife: 3%\n- Private Service: 3%\n- Government Service: 5%\n- Others: 3%\n\n#

In [39]:
output_text

['```json\n{\n  "textbox": {\n    "label": "Term Type",\n    "name": "Term Type"\n  },\n  "textbox": {\n    "label": "Rate Hold",\n    "name": "Rate Hold"\n  },\n  "textbox": {\n    "label": "LTV Min",\n    "name": "LTV Min"\n  },\n  "textbox": {\n    "label": "LTV Max",\n    "name": "LTV Max"\n  },\n  "textbox": {\n    "label": "TDS",\n    "name": "TDS"\n  },\n  "textbox": {\n    "label": "GDS",\n    "name": "GDS"\n  },\n  "textbox": {\n    "label": "Amortization",\n    "name": "Amortization"\n  },\n  "textbox": {\n    "label": "Property Value",\n    "name": "Property Value"\n  },\n  "textbox": {\n    "label": "Mortgage Amount",\n    "name": "Mortgage Amount"\n  },\n  "textbox": {\n    "label": "Rental Income Type",\n    "name": "Rental Income Type"\n  },\n  "textbox": {\n    "label": "Rental Income Value",\n    "name": "Rental Income Value"\n  },\n  "textbox": {\n    "label": "Beacon Min",\n    "name": "Beacon Min"\n  },\n  "textbox": {\n    "label": "Max Buydown",\n    "name": "Max 

In [48]:
output_text

['This bar chart is titled Which BI tools are used by respondents in the United Kingdom. The y-axis plots BI tools while the x-axis plots the percentage of respondents. The bar chart shows the percentage of respondents who use different BI tools.']